In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import datetime
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import requests

# funding company

In [ ]:
driver = webdriver.Chrome('/Users/hangy/Downloads/chromedriver')
driver.implicitly_wait(3)
driver.get('https://www.rocketpunch.com/investments/fund_companies?sort=invest_volume')
info = []

In [ ]:
for i in range(1,60):
    print(i)
    driver.find_element_by_xpath('//*[@id="search-results"]/div[2]/div/a[2]').click()
    driver.implicitly_wait(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') 
    contents = soup.find("div", {"id": "company-list"}).find_all("div", {"class": "content"})
    
    for content in contents:
        company = content.find('strong').text
        print(company)
        industry = content.find("span", {"class": "meta"}).text
        average = content.find_all("div", {"class": "content-eachline"})[0].find_all('span')[1].text
        total = content.find_all("div", {"class": "content-eachline"})[0].find_all('span')[3].text
        invest_list = content.find_all('img')
        invest = []
        for com in invest_list:
            invest.append(com.get('alt'))
        invest_num = content.find("a",{"class": "item"}).text
        invest_num = int(re.findall('\d+', invest_num)[0])

        temp = content.find('a').get('href')
        url = 'https://www.rocketpunch.com/'+ temp
        req = requests.get(url)
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')

        overview = soup.find("div", {"id": "company-overview"})
        if overview is None: overview = 'none'
        else : overview = overview.text
        
        fund_list = []
        funding_info = soup.find("section", {"id": "company-funding"}).find_all("div", {"class": "ui funding-item segment"})
        for fund in funding_info:
            fund_date = fund.find_all('span',{"class":'item'})[0].text
            fund_amount = fund.find('strong',{ "class":"amount"}) #투자금액
            if fund_amount is None : fund_amount= '비공개'
            else : fund_amount = fund_amount.text
            fund_value = fund.find('span',{ "class":"valuation"}).text #기업가치
            fund_header = fund.find_all('a',{'class':'header nowrap'})
            header = []
            for head in fund_header : 
                header.append(head.text)
            fund_list.append({'fund_date':fund_date,'fund_amount': fund_amount, 'fund_value':fund_value,'header':header})
        
        sub_information = soup.find("section", {"id": "company-info"})
        if sub_information is None : sub_info= ['none']
        else : 
            sub_info = []
            sub_information = soup.find("section", {"id": "company-info"}).find_all("div", {"class": "item"})
            for sub in sub_information:
                sub_info.append(re.sub('[\n]', '', sub.text).strip())
            
        category = []
        labels = soup.find_all("a", {"class": "ui circular basic label"})
        for label in labels:
            category.append(label.text)
        
        
        info.append({'company' : company, 'industry' : industry, 'average' : average, 'total': total ,
                     'invest_num' :invest_num, 'invest': invest,'overview': overview,
                     'funding_info':fund_list ,'sub_info':sub_info, 'category':category})
    

In [ ]:
info = pd.DataFrame(info)

In [ ]:
funding_company = info.drop_duplicates(['company'], keep='last')

In [ ]:
#funding_company.to_csv('rocket_funding_company_final.csv',  encoding = "utf-8-sig", header=True, index=False)

# funded company

In [ ]:
driver = webdriver.Chrome('/Users/hangy/Downloads/chromedriver')
driver.implicitly_wait(3)
driver.get('https://www.rocketpunch.com/investments/funded_companies?sort=funding_volume')
info = []

In [ ]:
for i in range(1,126):
    print(i)
    driver.find_element_by_xpath('//*[@id="search-results"]/div[2]/div/a[2]').click()
    driver.implicitly_wait(3)
    html = driver.page_source 
    soup = BeautifulSoup(html, 'html.parser') 
    contents = soup.find("div", {"id": "company-list"}).find_all("div", {"class": "content"})
    for content in contents:
        company = content.find('strong').text
        print(company)
        invest_num = content.find('a',{'class':'item'}).text
        invest_num = int(re.findall('\d+', invest_num)[0])

        temp = content.find('a').get('href')
        url = 'https://www.rocketpunch.com/'+ temp
        req = requests.get(url)
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')

        overview = soup.find("div", {"id": "company-overview"})
        if overview is None: overview = 'none'
        else : overview = overview.text
        
        fund_list = []
        funded_info = soup.find("section", {"id": "company-funded"}).find_all("div", {"class": "nowrap funding-content"})
        for fund in funded_info:
            fund_date = fund.find_all('span',{"class":'item'})[0].text
            fund_amount = fund.find('strong',{ "class":"amount"}) #투자금액
            if fund_amount is None : fund_amount= '비공개'
            else : fund_amount = fund_amount.text
            fund_value = fund.find('span',{ "class":"valuation"}).text #기업가치
            fund_header = fund.find_all('a',{'class':'header nowrap'})
            header = []
            for head in fund_header : 
                header.append(head.text)
            fund_list.append({'fund_date':fund_date,'fund_amount': fund_amount, 'fund_value':fund_value,'header':header})
        
        sub_info = []
        sub_information = soup.find("section", {"id": "company-info"}).find_all("div", {"class": "item"})
        for sub in sub_information:
            sub_info.append(re.sub('[\n]', '', sub.text).strip())
            
        category = []
        labels = soup.find_all("a", {"class": "ui circular basic label"})
        for label in labels:
            category.append(label.text)

        info.append({'company':company,'overview': overview, 'invest_num':invest_num,
                     'funded_info':fund_list ,'sub_info':sub_info, 'category':category})

In [ ]:
info = pd.DataFrame(info)

In [ ]:
funded_company = info.drop_duplicates(['company'], keep='last')

In [ ]:
#funded_company.to_csv('rocket_funded_company.csv',  encoding = "utf-8-sig", header=True, index=False)

# data preprocessing

In [20]:
path = r'C:/Users/hangy/Desktop/프로젝트/선보엔젤/5차결과물/' 

funded_company = 'rocket_funded_company_final.csv' 
funded_company = pd.read_csv(path+funded_company,encoding='utf-8-sig',engine='python')

funding_company = 'rocket_funding_company_final.csv'
funding_company = pd.read_csv(path+funding_company,encoding='utf-8-sig',engine='python')

In [21]:
print('funded_company list : ', len(funded_company))
print('funding_company list : ', len(funding_company))

funded_company list :  2499
funding_company list :  1164


In [16]:
funded_company

,category,company,funded_info,invest_num,overview,sub_info
0,"['모바일서비스', '웹서비스', '마케팅', '소프트웨어', '데이터', 'e-c...",쿠팡,"[{'fund_date': '2015-06-03', 'fund_amount': '\...",3,"""쿠팡은 세계에서 가장 빠르고 크게 성장하는 E-commerce 기업입니다. 우리는...","['설립일: 2010-07-01\xa0/\xa09년차', '..."
1,"['마케팅', '스타트업', 'startup']",KOTRA 수출창업지원팀,"[{'fund_date': '2008-12-18', 'fund_amount': '\...",1,국내 스타트업 기업의 글로벌 시장 진출 지원 \n\n,"['투자유치: 누적 6,000억 원 이상 상세보..."
2,['온라인게임'],넷마블게임즈,"[{'fund_date': '2014-03-26', 'fund_amount': '\...",1,none,"['투자유치: 누적 5,300억 원 이상 상세보..."
3,"['모바일광고', '모바일광고플랫폼', '애드테크', '애드네트워크', 'DSP',...",Avazu Korea,"[{'fund_date': '2017-07-28', 'fund_amount': '\...",1,글로벌 유니콘 스타트업이 한국에 진출합니다! About usDotC United G...,"['구성원: 501-1000명 상세보기', '투..."
4,['통신'],SK브로드밴드,"[{'fund_date': '2000-06-14', 'fund_amount': '\...",2,기업 소개가 없습니다. SK브로드밴드의 구성원이시면 구성원 등록 후 기업 소개를 작...,"['설립일: 1997-09-26\xa0/\xa022년차', ..."
5,['제조업'],서울반도체,"[{'fund_date': '2009-11-03', 'fund_amount': '\...",2,none,"['설립일: 1987-03-05\xa0/\xa032년차', ..."
6,"['유통', '전자제품']",롯데하이마트,"[{'fund_date': '2009-11-26', 'fund_amount': '\...",1,기업 소개가 없습니다. 롯데하이마트의 구성원이시면 구성원 등록 후 기업 소개를 작성...,"['설립일: 1999-01-01\xa0/\xa021년차', ..."
7,"['모바일서비스', '웹서비스', '소프트웨어', '모바일게임', '소셜커머스', ...",옐로모바일,"[{'fund_date': '2016-11-11', 'fund_amount': '\...",5,"옐로모바일은 '먹고', '마시고', '즐기는' 일상에서의 모바일 라이프 스타일을 지...","['설립일: 2012-08-01\xa0/\xa07년차', '..."
8,['제조업'],셀트리온,"[{'fund_date': '2010-04-22', 'fund_amount': '\...",1,기업 소개가 없습니다. 셀트리온의 구성원이시면 구성원 등록 후 기업 소개를 작성해 ...,"['설립일: 1991-02-27\xa0/\xa028년차', ..."
9,"['모바일서비스', '웹서비스', '마케팅', '소프트웨어', '이커머스', '소셜...",티몬,"[{'fund_date': '2017-04-27', 'fund_amount': '\...",4,"티몬은 '오프라인의 모든 것을 온라인으로 옮기자'는 비전을 가지고, 2010년에 설...","['설립일: 2010-05-10\xa0/\xa09년차', '..."


In [12]:
funded_company

,category,company,funded_info,invest_num,overview,sub_info
0,"['모바일서비스', '웹서비스', '마케팅', '소프트웨어', '데이터', 'e-c...",쿠팡,"[{'fund_date': '2015-06-03', 'fund_amount': '\...",3,"""쿠팡은 세계에서 가장 빠르고 크게 성장하는 E-commerce 기업입니다. 우리는...","['설립일: 2010-07-01\xa0/\xa09년차', '..."
1,"['마케팅', '스타트업', 'startup']",KOTRA 수출창업지원팀,"[{'fund_date': '2008-12-18', 'fund_amount': '\...",1,국내 스타트업 기업의 글로벌 시장 진출 지원 \n\n,"['투자유치: 누적 6,000억 원 이상 상세보..."
2,['온라인게임'],넷마블게임즈,"[{'fund_date': '2014-03-26', 'fund_amount': '\...",1,none,"['투자유치: 누적 5,300억 원 이상 상세보..."
3,"['모바일광고', '모바일광고플랫폼', '애드테크', '애드네트워크', 'DSP',...",Avazu Korea,"[{'fund_date': '2017-07-28', 'fund_amount': '\...",1,글로벌 유니콘 스타트업이 한국에 진출합니다! About usDotC United G...,"['구성원: 501-1000명 상세보기', '투..."
4,['통신'],SK브로드밴드,"[{'fund_date': '2000-06-14', 'fund_amount': '\...",2,기업 소개가 없습니다. SK브로드밴드의 구성원이시면 구성원 등록 후 기업 소개를 작...,"['설립일: 1997-09-26\xa0/\xa022년차', ..."
5,['제조업'],서울반도체,"[{'fund_date': '2009-11-03', 'fund_amount': '\...",2,none,"['설립일: 1987-03-05\xa0/\xa032년차', ..."
6,"['유통', '전자제품']",롯데하이마트,"[{'fund_date': '2009-11-26', 'fund_amount': '\...",1,기업 소개가 없습니다. 롯데하이마트의 구성원이시면 구성원 등록 후 기업 소개를 작성...,"['설립일: 1999-01-01\xa0/\xa021년차', ..."
7,"['모바일서비스', '웹서비스', '소프트웨어', '모바일게임', '소셜커머스', ...",옐로모바일,"[{'fund_date': '2016-11-11', 'fund_amount': '\...",5,"옐로모바일은 '먹고', '마시고', '즐기는' 일상에서의 모바일 라이프 스타일을 지...","['설립일: 2012-08-01\xa0/\xa07년차', '..."
8,['제조업'],셀트리온,"[{'fund_date': '2010-04-22', 'fund_amount': '\...",1,기업 소개가 없습니다. 셀트리온의 구성원이시면 구성원 등록 후 기업 소개를 작성해 ...,"['설립일: 1991-02-27\xa0/\xa028년차', ..."
9,"['모바일서비스', '웹서비스', '마케팅', '소프트웨어', '이커머스', '소셜...",티몬,"[{'fund_date': '2017-04-27', 'fund_amount': '\...",4,"티몬은 '오프라인의 모든 것을 온라인으로 옮기자'는 비전을 가지고, 2010년에 설...","['설립일: 2010-05-10\xa0/\xa09년차', '..."
